In [6]:
import torch

In [1]:
import os
Root = "/content/drive/MyDrive/shubham/gpt_dataset/"
os.chdir(Root)

In [3]:
!pip install transformers -q


     |████████████████████████████████| 5.5 MB 15.0 MB/s 
     |████████████████████████████████| 7.6 MB 70.6 MB/s 
     |████████████████████████████████| 182 kB 86.5 MB/s 


In [7]:
device = torch.device("cuda") # use GPU to train

In [4]:
import pandas as pd

In [8]:
data = pd.read_csv('imdb_sample/texts.csv')

In [9]:
data = data['text']

In [10]:
data.head()

0    Un-bleeping-believable! Meg Ryan doesn't even ...
1    This is a extremely well-made film. The acting...
2    Every once in a long while a movie will come a...
3    Name just says it all. I watched this movie wi...
4    This movie succeeds at being one of the most u...
Name: text, dtype: object

In [11]:
from sklearn.model_selection import train_test_split
train_ids, test_ids = train_test_split(data, test_size=0.2, random_state=42)


In [12]:
train_ids = pd.DataFrame(train_ids)

In [32]:
train_ids.head()

,text
29,"I have to be honest, i was expecting a failure..."
535,"I acquired this, one of my all-time favourite ..."
695,"Hmmm, yeah this episode is extremely underrate..."
557,"How does an usual day start in Point Place, Wi..."
836,I'm not sure under what circumstances director...


In [13]:
test_ids = pd.DataFrame(test_ids)

In [14]:
train_ids.to_csv("train.csv", index =False)
test_ids.to_csv("test.csv", index =False)

In [15]:
type(train_ids)

pandas.core.frame.DataFrame

In [16]:
from transformers import (
    GPT2Tokenizer,
    DataCollatorForLanguageModeling,
    TextDataset,
    GPT2LMHeadModel,
    TrainingArguments,
    Trainer,
    pipeline)


In [17]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [18]:
print('vocabulary size: %d, max squence length: %d' % (tokenizer.vocab_size, tokenizer.model_max_length))
print('tokenize sequence "Once upon a time in a little village":', tokenizer('Once upon a time in a little village'))


vocabulary size: 50257, max squence length: 1024
tokenize sequence "Once upon a time in a little village": {'input_ids': [7454, 2402, 257, 640, 287, 257, 1310, 7404], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


In [19]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


In [20]:
train_path="/content/drive/MyDrive/shubham/gpt_dataset/train.csv"
test_path ="/content/drive/MyDrive/shubham/gpt_dataset/test.csv"

In [21]:
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=train_path,
    block_size=128)
     
test_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=test_path,
    block_size=128)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
# train_dataset

In [22]:
print(tokenizer.decode(train_dataset[5]))

 she comes along, then Red stops yelling and kindly talks to her, making Eric feel left out of the family.<br /><br />Once this daily (painful) ritual is over, Eric rushes down to his basement, where all his friends are already hanging out. And when we get to see them, it becomes obvious Eric and his redhead tomboy girlfriend, next-door neighbor and childhood friend Donna Pinciotti are the sanest people around. Meet Steven Hyde, the conspiracy theorist who hates disco and doesn't really care about what's around as long as it's not funny to watch; Michael Kelso, the kind


In [24]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [25]:
model = model.to(device)

In [51]:
# training_args = TrainingArguments(
#     output_dir = 'model_data/out', # the output directory for the model predictions and checkpoints
#     overwrite_output_dir = True, # overwrite the content of the output directory
#     per_device_train_batch_size = 32, # the batch size for training
#     per_device_eval_batch_size = 32, # the batch size for evaluation
#     learning_rate = 5e-5, # defaults to 5e-5
#     num_train_epochs = 10, # total number of training epochs to perform
# )



training_args = TrainingArguments(
    output_dir="model_data/out", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=10, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [50]:
trainer = Trainer(
    model = model,
    args = training_args,
    data_collator=data_collator,
    train_dataset = train_dataset,
    eval_dataset = test_dataset
)

In [48]:
# trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2000
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 630
  Number of trainable parameters = 124439808


Step,Training Loss
500,3.382800


Saving model checkpoint to data/out/checkpoint-500
Configuration saved in data/out/checkpoint-500/config.json
Model weights saved in data/out/checkpoint-500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=630, training_loss=3.3409472268725198, metrics={'train_runtime': 163.054, 'train_samples_per_second': 122.659, 'train_steps_per_second': 3.864, 'total_flos': 1306460160000000.0, 'train_loss': 3.3409472268725198, 'epoch': 10.0})

In [52]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2000
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 630
  Number of trainable parameters = 124439808


Step,Training Loss
500,3.017000




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=630, training_loss=2.9783321320064484, metrics={'train_runtime': 158.8733, 'train_samples_per_second': 125.886, 'train_steps_per_second': 3.965, 'total_flos': 1306460160000000.0, 'train_loss': 2.9783321320064484, 'epoch': 10.0})

In [53]:


training_args = TrainingArguments(
    output_dir="model_data/out", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=20, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )

trainer = Trainer(
    model = model,
    args = training_args,
    data_collator=data_collator,
    train_dataset = train_dataset,
    eval_dataset = test_dataset
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [54]:
trainer.train()

***** Running training *****
  Num examples = 2000
  Num Epochs = 20
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1260
  Number of trainable parameters = 124439808


Step,Training Loss
500,2.570300
1000,2.294800


Saving model checkpoint to model_data/out/checkpoint-800
Configuration saved in model_data/out/checkpoint-800/config.json
Model weights saved in model_data/out/checkpoint-800/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1260, training_loss=2.359608435252356, metrics={'train_runtime': 321.6307, 'train_samples_per_second': 124.366, 'train_steps_per_second': 3.918, 'total_flos': 2612920320000000.0, 'train_loss': 2.359608435252356, 'epoch': 20.0})

In [55]:


training_args = TrainingArguments(
    output_dir="model_data/out", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=30, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )

trainer = Trainer(
    model = model,
    args = training_args,
    data_collator=data_collator,
    train_dataset = train_dataset,
    eval_dataset = test_dataset
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [56]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2000
  Num Epochs = 30
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1890
  Number of trainable parameters = 124439808


Step,Training Loss
500,1.820400
1000,1.542700
1500,1.343300


Saving model checkpoint to model_data/out/checkpoint-800
Configuration saved in model_data/out/checkpoint-800/config.json
Model weights saved in model_data/out/checkpoint-800/pytorch_model.bin
Saving model checkpoint to model_data/out/checkpoint-1600
Configuration saved in model_data/out/checkpoint-1600/config.json
Model weights saved in model_data/out/checkpoint-1600/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1890, training_loss=1.4964883471292163, metrics={'train_runtime': 485.3867, 'train_samples_per_second': 123.613, 'train_steps_per_second': 3.894, 'total_flos': 3919380480000000.0, 'train_loss': 1.4964883471292163, 'epoch': 30.0})

In [57]:

training_args = TrainingArguments(
    output_dir="model_data/out", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=40, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )

trainer = Trainer(
    model = model,
    args = training_args,
    data_collator=data_collator,
    train_dataset = train_dataset,
    eval_dataset = test_dataset
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [58]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2000
  Num Epochs = 40
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2520
  Number of trainable parameters = 124439808


Step,Training Loss
500,0.984800
1000,0.759500
1500,0.666000
2000,0.598200
2500,0.545400


Saving model checkpoint to model_data/out/checkpoint-800
Configuration saved in model_data/out/checkpoint-800/config.json
Model weights saved in model_data/out/checkpoint-800/pytorch_model.bin
Saving model checkpoint to model_data/out/checkpoint-1600
Configuration saved in model_data/out/checkpoint-1600/config.json
Model weights saved in model_data/out/checkpoint-1600/pytorch_model.bin
Saving model checkpoint to model_data/out/checkpoint-2400
Configuration saved in model_data/out/checkpoint-2400/config.json
Model weights saved in model_data/out/checkpoint-2400/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2520, training_loss=0.7093428570126731, metrics={'train_runtime': 651.5461, 'train_samples_per_second': 122.785, 'train_steps_per_second': 3.868, 'total_flos': 5225840640000000.0, 'train_loss': 0.7093428570126731, 'epoch': 40.0})

In [60]:
trainer.save_model("/content/drive/MyDrive/shubham/gpt_dataset/model")

Saving model checkpoint to /content/drive/MyDrive/shubham/gpt_dataset/model
Configuration saved in /content/drive/MyDrive/shubham/gpt_dataset/model/config.json
Model weights saved in /content/drive/MyDrive/shubham/gpt_dataset/model/pytorch_model.bin


# Testing the model

In [61]:
generator = pipeline('text-generation', tokenizer='gpt2', model='/content/drive/MyDrive/shubham/gpt_dataset/model')


loading configuration file /content/drive/MyDrive/shubham/gpt_dataset/model/config.json
Model config GPT2Config {
  "_name_or_path": "/content/drive/MyDrive/shubham/gpt_dataset/model",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_ve

In [39]:
train_ids['text'][3]

'Name just says it all. I watched this movie with my dad when it came out and having served in Korea he had great admiration for the man. The disappointing thing about this film is that it only concentrate on a short period of the man\'s life - interestingly enough the man\'s entire life would have made such an epic bio-pic that it is staggering to imagine the cost for production.<br /><br />Some posters elude to the flawed characteristics about the man, which are cheap shots. The theme of the movie "Duty, Honor, Country" are not just mere words blathered from the lips of a high-brassed officer - it is the deep declaration of one man\'s total devotion to his country.<br /><br />Ironically Peck being the liberal that he was garnered a better understanding of the man. He does a great job showing the fearless general tempered with the humane side of the man.'

In [41]:
#for 10 epoch
print(generator('Name just says it all', max_length=100)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Name just says it all, it isn't worth giving up to the film!"
""The second part... is kind of a dull. I was going for a nice story of a married couple. Of course the second part, was great in the end.<br /><br />The one major weakness is the poor English. The French were excellent and the scenes were funny. The acting seems pretty good, too. Not sure if there are some serious problems, maybe the editing, or some technical


In [62]:
#for 110 epoch
print(generator('Name just says it all', max_length=100)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Name just says it all!"
"I was never a big fan of horror movies, but this one really sucked. I'm starting to like Chris Cooper now. His character is surprisingly believable and I haven't experienced much character growth in the first 3 films. But in the last 3 films, they just kind of build up and he just has a horrible time. The director Tudor Giurgiu tried to bring some personality to the role, but it is a big job and he's just a


In [43]:
#for 10 epoch
text_beam = generator('Name just says it all',
                      max_length=100,
                      num_beams=5)
print(text_beam[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Name just says it all. <br /><br />If you don't like this movie, you won't like this movie. If you don't like this movie, you won't like this movie. If you don't like this movie, you won't like this movie. If you don't like this movie, you won't like this movie. If you don't like this movie, you won't like this movie. If you don't like this movie, you won't like this


In [63]:
#for 110 epoch
text_beam = generator('Name just says it all',
                      max_length=100,
                      num_beams=5)
print(text_beam[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Name just says it all!"
"I have seen many, many productions of The Nutcracker. Now perhaps I viewed this movie from the tainted point of view of a theatrical director, but I was disappointed. I'm sure people in the specific business of ballet choreography find this production impressive but from a purely theatrical perspective I found everything from design to choreography to be lackluster and unbefitting of a ""motion picture"". None of the traditionally ""weird"" and impressive costumes looked like


In [64]:
#for 110 epoch

text_random_sampling = generator('Name just says it all',
                                 max_length=100,
                                 top_k=0,
                                 do_sample=True,
                                 temperature=0.7)
print(text_random_sampling[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Name just says it all."
"This was a terrible film. I think the producer probably felt he couldn't get past the beginning of the film without rapping, which was pretty much the only thing worth discussing. This film didn't get the marks it seemed promised. It had the Jimmy Fallon aspects of a Jimmy Dean film but it was also completely devoid of comedy. Or like Fallon. At least the fashion model wasn't making out with Ben Affleck. At least that's what I heard


In [65]:
# for 110 epoch

text_k_sampling = generator('Name just says it all',
                            max_length=100,
                            top_k=40,
                            do_sample=True)
print(text_k_sampling[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Name just says it all! In the many films I have seen, Naughton never projects any spirit of the original plot. He plays the role of an elderly Friend, played by Preston Sturges, who essentially becomes a doormat when he awakens one day to find his room completely covered by rubble. He persistently attempts to socialize with Muriel, but finds she is very kind and kind, and will not allow him to stay alone. Little by little he gradually realizes that she


In [66]:
#for 110 epoch
text_p_sampling = generator('Name just says it all',
                            max_length=100,
                            top_k=0,
                            top_p=0.92,
                            do_sample=True)
print(text_p_sampling[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Name just says it all! I've seen some pretty bad movies, and this one stuck with me. It's a combo of The Arrival and Stanley Kubrick's ""Spielberg"" (which means nothing to me since it's not based on a book), so when I saw this it really sucked. The plot was exactly what I was hoping for, but alas it didn't. I could smile all day long, laugh at every minute of the movie (and some of that laughter actually made
